In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\public\anconda\envs\tf13\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

1.13.1
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.18.1
pandas 0.24.2
sklearn 0.22.1
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [3]:
fashion_mnist = keras.datasets.fashion_mnist
# print(fashion_mnist)
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]
# 打印格式
print(x_valid.shape,y_valid.shape)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [4]:
print(np.max(x_train),np.min(x_train))

255 0


In [5]:
# 数据归一化
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train:[None,28,28] -> [None,784]
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28*28)

In [6]:
print(np.max(x_train_scaled),np.min(x_train_scaled))

2.0231433 -0.8105136


In [7]:
hidden_units = [100,100]
class_num = 10

x = tf.placeholder(tf.float32,[None,28*28])
y = tf.placeholder(tf.int64,[None])
# 隐藏层
input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer,hidden_unit,activation=tf.nn.relu)

# 输出层
logits = tf.layers.dense(input_for_next_layer,class_num)
# last_hidden_output * w(logits) -> softmax -> pro
# 1. logit -> softmax -> prob
# 2. labels -> ont_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)

# get accuracy
prediction = tf.argmax(logits,1)
correct_prediction = tf.equal(prediction,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

# 训练
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [8]:
print(x)
print(logits)

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


In [9]:
x_train.shape

(55000, 28, 28)

In [ ]:
# session

init = tf.global_variables_initializer()
batch_size = 20
epochs = 10
train_steps_per_epoch = x_train.shape[0] // batch_size
valid_steps = x_valid.shape[0] // batch_size

def eval_with_sess(sess,x,y,accuracy,images,labels,batch_size):
    eval_steps = images.shape[0] // batch_size
    eval_accuracies = []
    for step in range(eval_steps):
        batch_data = images[step * batch_size : (step+1)*(batch_size)]
        batch_label = labels[step * batch_size: (step + 1)*batch_size]
        accuracy_val = sess.run(accuracy,feed_dict = {
            x:batch_data,
            y:batch_label
        })
        eval_accuracies.append(accuracy_val)
    return np.mean(eval_accuracies)

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            batch_data = x_train_scaled[step*batch_size:(step + 1)*(batch_size)]
            batch_label = y_train[step*batch_size:(step+1)*batch_size]
#             print(batch_label.shape)
#             print(batch_data.shape)
            loss_val,accuracy_val,_ = sess.run(
                [loss,accuracy,train_op],feed_dict = {
                    x:batch_data,
                    y:batch_label
                })
            print('\r[Train] epoch: %d,step: %d,loss:%3.5f,accuracy: %2.2f'%(epoch,step,loss_val,accuracy_val),end='')
            valid_accuracy = eval_with_sess(sess,x,y,accuracy,x_valid_scaled,y_valid,batch_size)
            print('\t[valid] acc : % 2.2f'%(valid_accuracy))

[Train] epoch: 0,step: 0,loss:2.48051,accuracy: 0.15	[valid] acc :  0.23
[Train] epoch: 0,step: 1,loss:2.37692,accuracy: 0.15	[valid] acc :  0.30
[Train] epoch: 0,step: 2,loss:2.13756,accuracy: 0.30	[valid] acc :  0.37
[Train] epoch: 0,step: 3,loss:2.20237,accuracy: 0.20	[valid] acc :  0.43
[Train] epoch: 0,step: 4,loss:1.57274,accuracy: 0.45	[valid] acc :  0.49
[Train] epoch: 0,step: 5,loss:1.68445,accuracy: 0.45	[valid] acc :  0.54
[Train] epoch: 0,step: 6,loss:1.57577,accuracy: 0.55	[valid] acc :  0.57
[Train] epoch: 0,step: 7,loss:1.47803,accuracy: 0.50	[valid] acc :  0.58
[Train] epoch: 0,step: 8,loss:1.33479,accuracy: 0.60	[valid] acc :  0.59
[Train] epoch: 0,step: 9,loss:1.38714,accuracy: 0.55	[valid] acc :  0.60
[Train] epoch: 0,step: 10,loss:1.21008,accuracy: 0.65	[valid] acc :  0.61
[Train] epoch: 0,step: 11,loss:1.17582,accuracy: 0.55	[valid] acc :  0.62
[Train] epoch: 0,step: 12,loss:2.22720,accuracy: 0.45	[valid] acc :  0.64
[Train] epoch: 0,step: 13,loss:1.28455,accuracy:

[Train] epoch: 0,step: 219,loss:0.40830,accuracy: 0.90	[valid] acc :  0.79
[Train] epoch: 0,step: 220,loss:0.67011,accuracy: 0.70	[valid] acc :  0.79
[Train] epoch: 0,step: 221,loss:0.79157,accuracy: 0.70	[valid] acc :  0.80
[Train] epoch: 0,step: 222,loss:1.00099,accuracy: 0.55	[valid] acc :  0.80
[Train] epoch: 0,step: 223,loss:0.46139,accuracy: 0.80	[valid] acc :  0.81
[Train] epoch: 0,step: 224,loss:0.50781,accuracy: 0.80	[valid] acc :  0.81
[Train] epoch: 0,step: 225,loss:0.52995,accuracy: 0.95	[valid] acc :  0.80
[Train] epoch: 0,step: 226,loss:0.77454,accuracy: 0.75	[valid] acc :  0.79
[Train] epoch: 0,step: 227,loss:0.39416,accuracy: 0.90	[valid] acc :  0.77
[Train] epoch: 0,step: 228,loss:0.58516,accuracy: 0.80	[valid] acc :  0.76
[Train] epoch: 0,step: 229,loss:0.68354,accuracy: 0.65	[valid] acc :  0.76
[Train] epoch: 0,step: 230,loss:0.60802,accuracy: 0.80	[valid] acc :  0.78
[Train] epoch: 0,step: 231,loss:0.81939,accuracy: 0.70	[valid] acc :  0.79
[Train] epoch: 0,step: 23

[Train] epoch: 0,step: 437,loss:0.55821,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 438,loss:0.47920,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 439,loss:0.60875,accuracy: 0.75	[valid] acc :  0.83
[Train] epoch: 0,step: 440,loss:0.21235,accuracy: 0.90	[valid] acc :  0.83
[Train] epoch: 0,step: 441,loss:0.50819,accuracy: 0.75	[valid] acc :  0.82
[Train] epoch: 0,step: 442,loss:0.39322,accuracy: 0.85	[valid] acc :  0.82
[Train] epoch: 0,step: 443,loss:0.39647,accuracy: 0.85	[valid] acc :  0.82
[Train] epoch: 0,step: 444,loss:0.77139,accuracy: 0.65	[valid] acc :  0.83
[Train] epoch: 0,step: 445,loss:0.41048,accuracy: 0.90	[valid] acc :  0.83
[Train] epoch: 0,step: 446,loss:0.56105,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 447,loss:0.64596,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 448,loss:0.48701,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 449,loss:0.47739,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 45

[Train] epoch: 0,step: 547,loss:0.59660,accuracy: 0.75	[valid] acc :  0.83
[Train] epoch: 0,step: 548,loss:0.58904,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 549,loss:0.40596,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 550,loss:0.62034,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 551,loss:0.92751,accuracy: 0.70	[valid] acc :  0.83
[Train] epoch: 0,step: 552,loss:0.35625,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 553,loss:0.22097,accuracy: 0.95	[valid] acc :  0.83
[Train] epoch: 0,step: 554,loss:0.42353,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 555,loss:0.54261,accuracy: 0.80	[valid] acc :  0.83
[Train] epoch: 0,step: 556,loss:0.25392,accuracy: 0.95	[valid] acc :  0.83
[Train] epoch: 0,step: 557,loss:0.30269,accuracy: 0.90	[valid] acc :  0.84
[Train] epoch: 0,step: 558,loss:0.32738,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 559,loss:0.49896,accuracy: 0.85	[valid] acc :  0.84
[Train] epoch: 0,step: 56

[Train] epoch: 0,step: 766,loss:0.34403,accuracy: 0.90	[valid] acc :  0.83
[Train] epoch: 0,step: 767,loss:0.42248,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 768,loss:0.18810,accuracy: 0.90	[valid] acc :  0.83
[Train] epoch: 0,step: 769,loss:0.30346,accuracy: 0.95	[valid] acc :  0.83
[Train] epoch: 0,step: 770,loss:0.52595,accuracy: 0.75	[valid] acc :  0.83
[Train] epoch: 0,step: 771,loss:0.72437,accuracy: 0.55	[valid] acc :  0.83
[Train] epoch: 0,step: 772,loss:0.31086,accuracy: 0.95	[valid] acc :  0.83
[Train] epoch: 0,step: 773,loss:0.29132,accuracy: 0.85	[valid] acc :  0.83
[Train] epoch: 0,step: 774,loss:0.62508,accuracy: 0.80	[valid] acc :  0.84
[Train] epoch: 0,step: 775,loss:0.17965,accuracy: 0.95	[valid] acc :  0.84
[Train] epoch: 0,step: 776,loss:0.28467,accuracy: 0.85	[valid] acc :  0.84
[Train] epoch: 0,step: 777,loss:0.95925,accuracy: 0.75	[valid] acc :  0.84
[Train] epoch: 0,step: 778,loss:0.36275,accuracy: 0.90	[valid] acc :  0.84
[Train] epoch: 0,step: 77

...

[Train] epoch: 8,step: 286,loss:0.16013,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 287,loss:0.19715,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 288,loss:0.26611,accuracy: 0.85	[valid] acc :  0.88
[Train] epoch: 8,step: 289,loss:0.19799,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 290,loss:0.27443,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 291,loss:0.51150,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 292,loss:0.04546,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 293,loss:0.13657,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 294,loss:0.16799,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 295,loss:0.23403,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 296,loss:0.26721,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 297,loss:0.14076,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 298,loss:0.37647,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 29

[Train] epoch: 8,step: 505,loss:0.24033,accuracy: 0.85	[valid] acc :  0.88
[Train] epoch: 8,step: 506,loss:0.50703,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 507,loss:0.18934,accuracy: 0.90	[valid] acc :  0.89
[Train] epoch: 8,step: 508,loss:0.04880,accuracy: 1.00	[valid] acc :  0.89
[Train] epoch: 8,step: 509,loss:0.30573,accuracy: 0.90	[valid] acc :  0.89
[Train] epoch: 8,step: 510,loss:0.29627,accuracy: 0.90	[valid] acc :  0.89
[Train] epoch: 8,step: 511,loss:0.40665,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 512,loss:0.07550,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 513,loss:0.19273,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 514,loss:0.36154,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 515,loss:0.13979,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 516,loss:0.08544,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 517,loss:0.24691,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 51

[Train] epoch: 8,step: 615,loss:0.13303,accuracy: 0.95	[valid] acc :  0.89
[Train] epoch: 8,step: 616,loss:0.45903,accuracy: 0.75	[valid] acc :  0.89
[Train] epoch: 8,step: 617,loss:0.16992,accuracy: 0.95	[valid] acc :  0.89
[Train] epoch: 8,step: 618,loss:0.28818,accuracy: 0.85	[valid] acc :  0.89
[Train] epoch: 8,step: 619,loss:0.09848,accuracy: 1.00	[valid] acc :  0.89
[Train] epoch: 8,step: 620,loss:0.11594,accuracy: 0.95	[valid] acc :  0.89
[Train] epoch: 8,step: 621,loss:0.10311,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 622,loss:0.29353,accuracy: 0.85	[valid] acc :  0.88
[Train] epoch: 8,step: 623,loss:0.23284,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 624,loss:0.11222,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 625,loss:0.21825,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 626,loss:0.58813,accuracy: 0.75	[valid] acc :  0.88
[Train] epoch: 8,step: 627,loss:0.11274,accuracy: 0.95	[valid] acc :  0.89
[Train] epoch: 8,step: 62

[Train] epoch: 8,step: 725,loss:0.05457,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 726,loss:0.20038,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 727,loss:0.30347,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 728,loss:0.20645,accuracy: 0.85	[valid] acc :  0.88
[Train] epoch: 8,step: 729,loss:0.18959,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 730,loss:0.07138,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 731,loss:0.05199,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 732,loss:0.09610,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 733,loss:0.12705,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 734,loss:0.02621,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 735,loss:0.41619,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 736,loss:0.06806,accuracy: 1.00	[valid] acc :  0.89
[Train] epoch: 8,step: 737,loss:0.06406,accuracy: 1.00	[valid] acc :  0.89
[Train] epoch: 8,step: 73

[Train] epoch: 8,step: 835,loss:0.13644,accuracy: 0.95	[valid] acc :  0.88
[Train] epoch: 8,step: 836,loss:0.11669,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 837,loss:0.50435,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 838,loss:0.28389,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 839,loss:0.26968,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 840,loss:0.26613,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 841,loss:0.13728,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 842,loss:0.44749,accuracy: 0.85	[valid] acc :  0.88
[Train] epoch: 8,step: 843,loss:0.16771,accuracy: 0.90	[valid] acc :  0.88
[Train] epoch: 8,step: 844,loss:0.05497,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 845,loss:0.30275,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 846,loss:0.02138,accuracy: 1.00	[valid] acc :  0.88
[Train] epoch: 8,step: 847,loss:0.34974,accuracy: 0.80	[valid] acc :  0.88
[Train] epoch: 8,step: 84